In [3]:
import pyes
import elasticsearch

es_address='localhost:9200'
conn = pyes.es.ES(es_address)
bq = pyes.query.BoolQuery() 
#q = pyes.query.MatchAllQuery()

tq = pyes.query.TermQuery(field="actor_email", value="ericmao@cyber00rn.org")

ESR = pyes.ESRange(field="id_time", from_value="2016-03-01", to_value="2016-04-30", 
                   include_lower=True ,include_upper=False)
rq = pyes.query.RangeQuery(qrange=ESR)

bq.add_must(tq)
bq.add_must(rq)

result = conn.search(query=bq , indices='google' , doc_types='drive') 
len(result)

5672

In [4]:
country_set=set()
for i in range(0,100):
    if 'geoip_countryCode' in result[i]:
        country_set.add(result[i]['geoip_countryCode'])

len(country_set)
country_set

ip_set=set()
for i in range(0,100):
    if 'ipAddress' in result[i]:
        ip_set.add(result[i]['ipAddress']+","+result[i]['geoip_countryCode'])

len(ip_set)
ip_set

geo_set=set()
for i in range(0,len(result)):
    if 'ipAddress' in result[i]:
        geo_set.add(str(result[i]['geoip_latitude'])+","+str(result[i]['geoip_longitude']))

len(geo_set)
geo_set

{'23.5,121',
 '25.039199829101562,121.52499389648438',
 '37.56709289550781,-122.37100219726562',
 '37.77490234375,-122.41940307617188',
 '38,-97',
 '50.08329772949219,14.416702270507812',
 '50.211700439453125,15.844192504882812'}

In [5]:
result[0]

{'id_applicationName': 'drive',
 'actor_profileId': '108189844280408676879',
 'geoip_countryName': 'Taiwan',
 'event_old_value': ['none'],
 'kind': 'admin#reports#activity',
 'geoip_latitude': 25.039199829101562,
 'event_doc_title': 'E0FEFDC2-7667-4AD3-AA88-737B330AF6F0.jpg',
 'id_uniqueQualifier': 8195853153144543330,
 'ipAddress': '211.73.74.35',
 'geoip_countryCode': 'TW',
 'actor_email': 'ericmao@cyber00rn.org',
 'event_new_value': ['owner'],
 'event_doc_id': '0BwHnaad0ivbsVHIzRUhCd3FScFk',
 'event_doc_type': 'unknown',
 'id_time': '2016-04-27T01:56:40.960Z',
 'event_type': 'acl_change',
 'event_primary_event': False,
 'event_owner': 'ericmao@cyber00rn.org',
 'geoip_city': 'Taipei',
 'event_name': 'change_user_access',
 'id_customerId': 'C02u7zbcq',
 'geoip_longitude': 121.52499389648438,
 'event_target_user': 'ericmao@cyber00rn.org'}

In [6]:
import json

conn = pyes.es.ES('localhost:9200')
q = pyes.query.MatchAllQuery()
tagg = pyes.aggs.TermsAgg('actor_email', field= 'actor_email') 

qsearch = pyes.query.Search(q) 

qsearch.agg.add(tagg) 

rs = conn.search(query=qsearch,index='google',type="drive") 
print json.dumps(rs.aggs,indent=2)

SyntaxError: invalid syntax (<ipython-input-6-13d8c19da625>, line 12)

In [ ]:
import pyes
import json
conn=pyes.es.ES('localhost:9200')
q = pyes.MatchAllQuery()
tagg = pyes.aggs.TermsAgg('user', field= 'actor_email', sub_aggs=[]) 
tagg1 = pyes.aggs.TermsAgg('country', field= 'geoip_countryName')  
tagg.sub_aggs.append(tagg1) 
# 將tagg1加到tagg.sub_aggs裡面。
qsearch = pyes.query.Search(q) 
# This "Search" is under pyes.query, http://pydoc.net/Python/pyes/0.99.5/pyes.query
qsearch.agg.add(tagg)

rs = conn.search(query=qsearch , indices='google' ,type="drive" )
print json.dumps(rs.aggs,indent=2)

In [ ]:
qsearch